# Data Wrangling WeRateDogs

## Gathering

* **Twitter archive file** Manually download by clicking the link: 
* **Tweet Image prediction** Programtically download the file 
* **Tweet JSON file** Get from twitter using API. 

In [3]:
#Imports 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline 
import requests 
import tweepy
import json
import time

### 1 Twitter archive 

In [4]:
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

### 2 Tweet Image prediction

In [5]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
page = requests.get(url)
with open('image_predictions.tsv', mode = 'wb') as f: 
    f.write(page.content)
image_prediction = pd.read_csv('image_predictions.tsv', sep = '\t')

### 3 Tweet JSON file

In [6]:
consumer_key = 'IlUpuNICtvcgHR66TrBLN43k7'
consumer_secret = 'WZXOLRl7sBUpfIBeoukENQ5Z0ZcfPUOB6ouyxSrTYwNcRTXhSs'
access_token = '1307275021522530306-jmWn1Fm4mu7i431hpL4gnMsd3dTCfY'
access_secret = 'NtlNyDFUgIPFcTxlFDhEPzSeVRAv1vtQLm61dDIBPzFZ5'

In [7]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [8]:
#Create a list of the tweet ids from the image_prediction DataFrame. 
tweet_ids = list(image_prediction['tweet_id'])

In [9]:
#quick test to check if we can pull tweets correctly. 
tweet = api.get_status(tweet_ids[0], tweet_mode = 'extended')
print(tweet.full_text)

Here we have a Japanese Irish Setter. Lost eye in Vietnam (?). Big fan of relaxing on stair. 8/10 would pet https://t.co/BLDqew2Ijj


In [17]:
#Create an empty dictionnary and fill it in with the following columns: tweet_id, retweet_count, favorite_count
dog_dict = {}
dog_dict['favorite_count'], dog_dict['retweet_count'], dog_dict['tweet_id'], dog_dict['created_at'] = [], [], [], []
start = time.time()

for tweet_id in tweet_ids:
    try:
        tweet = api.get_status(tweet_id, tweet_mode = 'extended')
        
        dog_dict['favorite_count'].append(tweet.favorite_count)
        
        dog_dict['retweet_count'].append(tweet.retweet_count)
        
        dog_dict['tweet_id'].append(tweet.id)
                                   
        dog_dict['created_at'].append(tweet.created_at)
        
    except Exception as e:
        print(e)
    
end = time.time()
print('time taken: {}s '.format(end-start))

[{'code': 144, 'message': 'No status found with that ID.'}]


Rate limit reached. Sleeping for: 307


[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 179, 'message': 'Sorry, you are not authorized to see this status.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/statuses/show.json?id=793135492858580992&tweet_mode=extended (Caused by SSLError(SSLError("bad handshake: SysCallError(50, 'ENETDOWN')")))
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'cod

In [10]:
#Used stackoverflow to overcome: 'datetime.datetime is not JSON serializable.'
with open('tweet_json.txt', 'w') as f:
    json.dump(dog_dict, f, indent =4, sort_keys = True, default = str)

NameError: name 'dog_dict' is not defined

In [13]:
#Create a DataFrame object from the json file.
tweet_df = pd.read_json('tweet_json.txt')

ValueError: Expected object or value

In [ ]:
#Imports 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline 
import requests 
import tweepy
import json
import time

In [ ]:
tweet_df.sample(10)

In [ ]:
image_prediction.sample(10)

In [ ]:
twitter_archive.sample(10)

### Programmatic assessment

In [11]:
tweet_df.info()

NameError: name 'tweet_df' is not defined

In [ ]:
tweet_df.describe()

In [ ]:
image_prediction.info()

In [ ]:
image_prediction.describe()

In [ ]:
twitter_archive.info()

In [ ]:
twitter_archive.describe()

### Main issues

#### Quality issues 
* twitter_archive: source column is in HTML format.
* image_prediction: some dog names in p1, p2 and p3 start with capitals and others don't. 
* 
* 
* 
* 
* 
* 

#### Tidiness issues

* twitter_archive : 4 columns for the different dog stages, when only one is necessary.
* 

In [ ]:
#first, create copies of the DataFrames
twitter_archive_clean = twitter_archive.copy()
image_prediction_clean = image_prediction.copy()
tweet_df_clean = tweet_df.copy()

In [ ]:
#twitter_archive_clean['source'] = twitter_archive_clean['source'].str.replace('href=', '').replace('<a', '')

In [ ]:
twitter_archive_clean['source'][1]

*twitter_archive : 4 columns for the different dog stages, when only one is necessary.*
> Create a new column dog_stage which melts these 4 stages. Then drop the columns.

In [ ]:
twitter_archive_clean.columns

In [ ]:
melt_cols = ['doggo', 'floofer', 'pupper', 'puppo']
kept_cols = [c for c in list(twitter_archive_clean.columns) if c not in melt_cols]

In [ ]:
twitter_archive_clean = pd.melt(twitter_archive_clean, id_vars = kept_cols, value_vars= melt_cols, var_name = 'stages', value_name ='dog_stage')
twitter_archive_clean.drop('stages', axis= 1, inplace= True)

##### Check

In [ ]:
twitter_archive_clean.head()